<a href="https://colab.research.google.com/github/1hamzaiqbal/MFCLIP_acv/blob/hanson%2Fsiglip/hanson_siglip_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --branch hanson/siglip https://github.com/1hamzaiqbal/MFCLIP_acv.git
!pip install torch torchvision timm einops yacs tqdm opencv-python scikit-learn scipy pyyaml ruamel.yaml pytorch-ignite foolbox pandas matplotlib seaborn wilds ftfy

#google drive
from google.colab import drive
drive.mount('/content/drive')

Cloning into 'MFCLIP_acv'...
remote: Enumerating objects: 496, done.
remote: Counting objects: 100% (496/496), done.
remote: Compressing objects: 100% (366/366), done.
remote: Total 496 (delta 137), reused 464 (delta 113), pack-reused 0 (from 0)
Receiving objects: 100% (496/496), 3.45 MiB | 13.27 MiB/s, done.
Resolving deltas: 100% (137/137), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.8/343.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.1/753.1 kB 52.7 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/c

In [ ]:
%%shell
cd MFCLIP_acv/
mkdir -p logs/baseline/
chmod +x ./scripts/baseline.sh

In [ ]:
#download oxford pets to drive
%%shell
wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
tar -xf /content/images.tar.gz -C /content/drive/MyDrive/FL25Courses/CSE5619/adcv_proj_data/oxford_pets

--2025-10-31 04:01:36--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://thor.robots.ox.ac.uk/pets/images.tar.gz [following]
--2025-10-31 04:01:37--  https://thor.robots.ox.ac.uk/pets/images.tar.gz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791918971 (755M) [application/octet-stream]
Saving to: ‘images.tar.gz.2’

images.tar.gz.2     100%[===================>] 755.23M  28.2MB/s    in 28s     

2025-10-31 04:02:05 (27.3 MB/s) - ‘images.tar.gz.2’ saved [791918971/791918971]



In [ ]:
#download oxford pets annotaitons to drive
%%shell
wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
tar -xf /content/annotations.tar.gz -C /content/drive/MyDrive/FL25Courses/CSE5619/adcv_proj_data/oxford_pets

--2025-10-31 04:05:42--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://thor.robots.ox.ac.uk/pets/annotations.tar.gz [following]
--2025-10-31 04:05:43--  https://thor.robots.ox.ac.uk/pets/annotations.tar.gz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19173078 (18M) [application/octet-stream]
Saving to: ‘annotations.tar.gz’

annotations.tar.gz  100%[===================>]  18.28M  12.7MB/s    in 1.4s    

2025-10-31 04:05:45 (12.7 MB/s) - ‘annotations.tar.gz’ saved [19173078/19173078]



In [ ]:
#download the split_zhou_OxfordPets.json required by the mf_clip repo
%%shell
gdown --id 1501r8Ber4nNKvmlFVQZ8SeUHTcdTTEqs -O /content/split_zhou_OxfordPets.json

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1501r8Ber4nNKvmlFVQZ8SeUHTcdTTEqs
To: /content/split_zhou_OxfordPets.json
100% 724k/724k [00:00<00:00, 50.6MB/s]


In [ ]:
#move downlaoded json file to drive
import shutil
shutil.copy("/content/split_zhou_OxfordPets.json", "/content/drive/MyDrive/FL25Courses/CSE5619/adcv_proj_data/oxford_pets/split_zhou_OxfordPets.json")

'/content/drive/MyDrive/FL25Courses/CSE5619/adcv_proj_data/oxford_pets/split_zhou_OxfordPets.json'

Finetune ArcFaceSigmoid on oxford pets

In [ ]:
# go to the repo root first
%cd /content/MFCLIP_acv

# (optional) quiet TF spam
%env TF_CPP_MIN_LOG_LEVEL=2

# run with repo-relative config paths (no MFCLIP_acv/ prefix)
!python main.py \
  --flag finetune \
  --dataset oxford_pets \
  --root  /content/drive/MyDrive/FL25Courses/CSE5619/adcv_proj_data\
  --config-file configs/trainers/CoOp/rn50.yaml \
  --dataset-config-file configs/datasets/oxford_pets.yaml \
  --trainer ZeroshotCLIP \
  --surrogate RN50 \
  --head ArcFaceSigmoid \
  --num_epoch 100 \
  --bs 64 \
  --lr 0.01 \
  --optimizer SGD \
  --ratio 0.2 \
  --device cuda:0

/content/MFCLIP_acv
env: TF_CPP_MIN_LOG_LEVEL=2
2025-10-31 16:54:19.132928: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761929659.153144    6614 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761929659.159601    6614 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761929659.175271    6614 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761929659.175299    6614 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761929659.175302    6614 c

Finetune SigLipHead on Oxford Pets

In [2]:
# go to the repo root first
%cd /content/MFCLIP_acv

# (optional) quiet TF spam
%env TF_CPP_MIN_LOG_LEVEL=2

# run with repo-relative config paths (no MFCLIP_acv/ prefix)
!python main.py \
  --flag finetune \
  --dataset oxford_pets \
  --root  /content/drive/MyDrive/FL25Courses/CSE5619/adcv_proj_data\
  --config-file configs/trainers/CoOp/rn50.yaml \
  --dataset-config-file configs/datasets/oxford_pets.yaml \
  --trainer ZeroshotCLIP \
  --surrogate RN50 \
  --head SigLipHead \
  --num_epoch 100 \
  --bs 64 \
  --lr 0.01 \
  --optimizer SGD \
  --ratio 0.2 \
  --device cuda:0

/content/MFCLIP_acv
env: TF_CPP_MIN_LOG_LEVEL=2
2025-10-31 19:36:12.536813: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761939372.559706     938 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761939372.566632     938 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761939372.583664     938 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761939372.583704     938 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761939372.583708     938 c

Eval Test

In [ ]:
# go to the repo root first
%cd /content/MFCLIP_acv

# (optional) quiet TF spam
%env TF_CPP_MIN_LOG_LEVEL=2


        # python main.py \
        # --root ${DATA} \
        # --seed ${SEED} \
        # --dataset-config-file configs/datasets/${DATASET}.yaml \
        # --config-file configs/trainers/CoOp/rn50.yaml \
        # --flag eval_adv \
        # --dataset ${DATASET} \
        # --attack ${at} \
        # > logs/eval/${DATASET}.log 2>&1


# run with repo-relative config paths (no MFCLIP_acv/ prefix)
!python main.py \
  --flag eval_adv \
  --dataset oxford_pets \
  --root  /content/drive/MyDrive/FL25Courses/CSE5619/adcv_proj_data\
  --config-file configs/trainers/CoOp/rn50.yaml \
  --dataset-config-file configs/datasets/oxford_pets.yaml \
  --surrogate RN50 \
  --head SigLipHead \